In [1]:
! pip3 install -U langchain-nomic retrying langchain_community tiktoken langchainhub chromadb langchain langgraph gpt4all firecrawl-py



[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [24]:
#this is to setup your langchain environment

#import os

#os.environ["LANGCHAIN_TRACING_V2"]="true"
#os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
#os.environ["LANGCHAIN_API_KEY"]="<use-your-langchain-api-key>"
#os.environ["LANGCHAIN_PROJECT"]="csulbgenai1"

In [2]:
local_llm = "llama3"

In [3]:
#setting up async for loading all crawled urls

%pip install --upgrade --quiet  nest_asyncio

# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()



[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)  # Disable warnings for SSL cert errors

# Function to get all CSULB links from a given URL
def get_all_links(url, session):
    try:
        response = session.get(url, timeout=10, verify=False)  # Added verify=False to bypass SSL cert errors
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Failed to fetch {url}: {e}")
        return set()

    soup = BeautifulSoup(response.content, 'html.parser')
    links = set()

    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        if 'csulb.edu' in href:
            full_url = urljoin(url, href)
            links.add(full_url)
    
    return links

# Function to crawl a single page
def crawl_page(url, depth, session):
    print(f"Crawling: {url} at depth {depth}")
    links = get_all_links(url, session)
    return url, links

# Main crawl function
def crawl(start_url, max_depth=2, max_workers=40):
    visited = set()
    to_visit = [(start_url, 0)]  # Each element is a tuple (URL, depth)

    # Setup requests session with retries
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount('http://', HTTPAdapter(max_retries=retries))
    session.mount('https://', HTTPAdapter(max_retries=retries))

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        while to_visit or futures:
            while to_visit and len(futures) < max_workers:
                current_url, depth = to_visit.pop()
                if depth <= max_depth and current_url not in visited:
                    future = executor.submit(crawl_page, current_url, depth, session)
                    futures.append(future)
                    visited.add(current_url)
            
            for future in as_completed(futures):
                futures.remove(future)
                try:
                    url, links = future.result()
                    for link in links:
                        if link not in visited:
                            to_visit.append((link, depth + 1))
                except Exception as e:
                    print(f"Error crawling page: {e}")

    return visited

if __name__ == "__main__":
    start_url = "http://catalog.csulb.edu/index.php?catoid=10"  # this URL is for starting at the CSULB catalog
    crawled_urls = crawl(start_url, max_depth=2)  # depth indicates how many urls away from the root

    print(f"Crawled URLs: {len(crawled_urls)}")
    for url in crawled_urls:
        print(url)


Crawling: http://catalog.csulb.edu/index.php?catoid=10 at depth 0
Crawling: https://www.csulb.edu/college-of-business at depth 1Crawling: https://www.csulb.edu/site-index at depth 1

Crawling: https://www.csulb.edu/college-of-health-human-services at depth 1
Crawling: https://www.csulb.edu/college-of-education at depth 1
Crawling: https://www.csulb.edu/student-affairs/counseling-and-psychological-services at depth 1
Crawling: http://web.csulb.edu/divisions/students/shs/ at depth 1
Crawling: https://www.csulb.edu/college-of-natural-sciences-and-mathematics at depth 1
Crawling: https://www.csulb.edu/college-of-engineering at depth 1
Crawling: https://cla.csulb.edu/ at depth 1
Crawling: http://catalog.csulb.edu at depth 1
Crawling: http://www.csulb.edu at depth 1
Crawling: https://sso.csulb.edu/ at depth 1
Crawling: http://web.csulb.edu/divisions/aa/catalog at depth 1
Crawling: https://www.csulb.edu/about-csulb at depth 1
Crawling: https://www.csulb.edu/university-police at depth 1
Crawli

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Crawling: http://www.csulb.edu/49er-shops-at-the-beach at depth 2Crawling: https://www.csulb.edu/cleryasr at depth 2

Crawling: https://www.csulb.edu/student-affairs/campus-rules-and-regulations/regulation-i at depth 2
Crawling: http://www.csulb.edu/administration-finance?utm_source=website&utm_medium=homepage&utm_content=menulink&utm_campaign=JumboMenu at depth 2
Crawling: https://cla.csulb.edu/academics/undergraduate-studies/cla-academic-majors/ at depth 2
Crawling: https://cla.csulb.edu/student-success-story-robin-gaitan/ at depth 2
Crawling: http://www.csulb.edu/help?utm_source=website&utm_medium=homepage&utm_content=menulink&utm_campaign=JumboMenu at depth 2
Crawling: http://www.csulb.edu/alumni/volunteers at depth 2
Crawling: https://cla.csulb.edu/about/ at depth 2
Crawling: https://www.csulb.edu/employment?utm_source=website&utm_medium=homepage&utm_content=footerlink&utm_campaign=web at depth 2
Crawling: https://cla.csulb.edu/wp-content/uploads/2015/12/Classroom-Emergency-Proced

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to fetch https://www.csulb.edu/academic-affairs/program-review-and-assessment/wscuc/wscuc-accreditation: 404 Client Error: Not Found for url: https://www.csulb.edu/academic-affairs/program-review-and-assessment/wscuc/wscuc-accreditation


...

Failed to fetch http://www.csulb.edu/menu/administration?utm_source=website&utm_medium=homepage&utm_content=menulink&utm_campaign=JumboMenu: 403 Client Error: Forbidden for url: https://www.csulb.edu/administration
Crawling: https://cla.csulb.edu/cla-commencement-photos-2024/ at depth 2Crawling: https://cla.csulb.edu/faculty-publications-romance-german-russian-languages-and-literatures-2024/ at depth 2
Crawling: https://cla.csulb.edu/student-success-stories-archive/ at depth 2

Crawling: http://specialevents.csulb.edu/MasterCalendar/MasterCalendar.aspx at depth 2
Crawling: http://web.csulb.edu/divisions/students/?utm_source=website&utm_medium=homepage&utm_content=menulink&utm_campaign=JumboMenu at depth 2
Crawling: https://cla.csulb.edu/departments-and-programs/ at depth 2
Crawling: https://cla.csulb.edu/centers-and-institutes/ at depth 2
Crawling: http://www.cla.csulb.edu at depth 2
Crawling: https://www.cpace.csulb.edu/courses/degree-programs/master-of-arts-in-international-affairs a

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Crawling: https://www.csulb.edu/caps at depth 2Crawling: https://www.csulb.edu/gearup at depth 2

Crawling: mailto:Shelly.Collins@csulb.edu at depth 2
Failed to fetch mailto:Shelly.Collins@csulb.edu: No connection adapters were found for 'mailto:Shelly.Collins@csulb.edu'
Crawling: mailto:Rosa.Moreno@csulb.edu at depth 2
Crawling: https://www.csulb.edu/eop at depth 2
Crawling: mailto:Ferdinand.Arcinue@csulb.edu at depth 2
Crawling: mailto:Abby.Bradecich@csulb.edu at depth 2
Failed to fetch mailto:Rosa.Moreno@csulb.edu: No connection adapters were found for 'mailto:Rosa.Moreno@csulb.edu'
Crawling: https://www.csulb.edu/student-affairs/university-outreach-school-relations/tours at depth 2
Failed to fetch mailto:Abby.Bradecich@csulb.edu: No connection adapters were found for 'mailto:Abby.Bradecich@csulb.edu'
Crawling: https://www.csulb.edu/guardian at depth 2
Failed to fetch mailto:Ferdinand.Arcinue@csulb.edu: No connection adapters were found for 'mailto:Ferdinand.Arcinue@csulb.edu'
Crawl

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Crawling: https://web.csulb.edu/divisions/urd/beachfacts/ at depth 2
Crawling: https://www.csulb.edu/social-media/directory at depth 2
Crawling: https://www.csulb.edu/institutional-research-analytics at depth 2
Crawling: http://emergency.csulb.edu/ at depth 2
Crawling: http://daf.csulb.edu/offices/its/catalog/directory.php at depth 2
Crawling: http://www.csulb.edu/about/ at depth 2
Crawling: http://www.ccpe.csulb.edu/ContinuingEd/default.aspx at depth 2
Crawling: http://www.ced.csulb.edu/ at depth 2
Crawling: http://www.csulb.edu/sitemap at depth 2
Crawling: http://www.csulb.edu/feedback/ at depth 2
Crawling: http://web.csulb.edu/colleges/coe/ at depth 2
Crawling: mailto:catalog@csulb.edu at depth 2
Failed to fetch mailto:catalog@csulb.edu: No connection adapters were found for 'mailto:catalog@csulb.edu'
Crawling: http://web.csulb.edu/colleges/cla/ at depth 2
Crawling: http://web.csulb.edu/colleges/cba/ at depth 2
Crawling: http://web.csulb.edu/colleges/chhs/ at depth 2
Crawling: http:

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(list(crawled_urls), continue_on_failure=True)
loader.requests_per_second = 2
docs = loader.aload()

Fetching pages:   0%|          | 0/534 [00:00<?, ?it/s]Error fetching https://www.csulb.edu/shs with attempt 1/3: Cannot connect to host www.csulb.edu:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')]. Retrying...
Error fetching http://www.csulb.edu/alumni?utm_source=website&utm_medium=homepage&utm_content=menulink&utm_campaign=JumboMenu with attempt 1/3: Cannot connect to host www.csulb.edu:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')]. Retrying...
Fetching pages:   0%|          | 0/534 [00:00<?, ?it/s]


KeyboardInterrupt: 

Error fetching https://www.csulb.edu/shs with attempt 2/3: Cannot connect to host www.csulb.edu:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')]. Retrying...
Error fetching http://www.csulb.edu/alumni?utm_source=website&utm_medium=homepage&utm_content=menulink&utm_campaign=JumboMenu with attempt 2/3: Cannot connect to host www.csulb.edu:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')]. Retrying...
Error fetching https://www.csulb.edu/shs, skipping due to continue_on_failure=True
Error fetching http://www.csulb.edu/alumni?utm_source=website&utm_medium=homepage&utm_content=menulink&utm_campaign=JumboMenu, skipping due to continue_on_failure=True
Error fetching https://www.csulb.edu/student-affairs/student-orientation-advising-and-registration with attempt 

In [168]:
#Split documents

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(docs)



In [169]:
#storing into chroma db

from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_community.vectorstores import Chroma
from langchain_community.llms import llamacpp

embedding = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf")

vectorstore = Chroma.from_documents(
  documents=all_splits,
  collection_name="rag-chroma-1",
  embedding=embedding,
)

retriever = vectorstore.as_retriever()

In [170]:
#testing retrievers ability to get relevant documents

doc = retriever.get_relevant_documents("college of business")

for document in doc:
    document.page_content = document.page_content.replace("\n", "")
    document.page_content = document.page_content.replace("\xa0", "")

print(doc)

[Document(page_content='Business AdministrationProfessional and International EducationEngineeringLiberal ArtsHealth & Human ServicesNatural Sciences & MathematicsAdditional ResourcesAbout CSULBUniversity CareersUniversity PoliceCampus DirectoryStrategic CommunicationsDownload Document ReaderC S U L B Contact UsCalifornia State University Long Beach', metadata={'description': 'CSULB is a large, urban, comprehensive university in the 23-campus California State University system.', 'language': 'en', 'source': 'http://catalog.csulb.edu/index.php?catoid=10', 'title': 'California State University, Long Beach - Modern Campus Catalog™'}), Document(page_content='Business AdministrationProfessional and International EducationEngineeringLiberal ArtsHealth & Human ServicesNatural Sciences & MathematicsAdditional ResourcesAbout CSULBUniversity CareersUniversity PoliceCampus DirectoryStrategic CommunicationsDownload Document ReaderC S U L B Contact UsCalifornia State University Long Beach', metadat

In [171]:
#Retrieval Grader
#Checks if query is located within the urls (ChromaDB)
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

#LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)
prompt = PromptTemplate(
  template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance
   of a retrieved document to a user question. If the document contains keywords related to the user question, 
   grade it as relevant. It does not need ot be a stringent test. The goal is to filter out erroneous retrievals. \n
   Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
   Provide the binary score as a JSON with a single key 'score' and no preamble or explaination. \n
   <|eot_id|><|start_header_id|>user<|end_header_id|>
   Here is the retrieved document: \n\n {document} \n\n
   Here is the user question: {question} \n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
  input_variables=["question", "document"]
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "how do i get to class?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


In [193]:
#Generating Output, based on relevance

from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt 
prompt = PromptTemplate(
  template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks 
  for students attending California State University, Long Beach.
  Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say you don't know.
  Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
  Question: {question}
  Context: {context}
  Answer: <|eot_id|><start_header_id|>assistant<|end_header_id|>""",
  input_variables=["question", "document"]
)

llm = ChatOllama(model=local_llm, temperature=0)

# Post-processing
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "tell me about csulb"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

CSULB is a large, urban, comprehensive university in the 23-campus California State University system. It is characterized by exceptional academic programs, outstanding support services, dedicated faculty, and a diverse student population. The campus provides an inviting collegiate atmosphere as well as state-of-the-art facilities.


In [209]:
#manual feedback loop

from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt 
prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks 
    for students attending California State University, Long Beach.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say you don't know.
    Use three sentences maximum and keep the answer concise user
    Question: {question}
    Context: {context}
    Answer: <start_header_id|>assistant""",
    input_variables=["question", "document"]
)

# Initialize the LLM
local_llm = "llama3"
llm = ChatOllama(model=local_llm, temperature=0.9, top_p=0.95)  # Adjust parameters for diversity

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Function to generate two responses
def generate_two_responses(question, docs):
    response1 = rag_chain.invoke({"context": docs, "question": question})
    response2 = rag_chain.invoke({"context": docs, "question": question})
    return response1, response2

# Run
question = "tell me about csulb"
docs = retriever.invoke(question)
response1, response2 = generate_two_responses(question, docs)

# Split responses into options
response_options1 = response1.split("\n\n")
option1 = response_options1[0].strip() if response_options1 else "No response generated."

response_options2 = response2.split("\n\n")
option2 = response_options2[0].strip() if response_options2 else "No response generated."

# Print the options
print("Option 1:")
print(option1)
print("\nOption 2:")
print(option2)

# User interaction
user_choice = input("Enter 1 to choose Option 1, 2 to choose Option 2, or any other key to skip: ")

# Handle user choice
if user_choice == '1' and option1 != "No response generated.":
    chosen_option = option1
    print(f"\nChosen response: {chosen_option}")
elif user_choice == '2' and option2 != "No response generated.":
    chosen_option = option2
    print(f"\nChosen response: {option2}")
else:
    print("\nInvalid choice or option not available.")


Option 1:
Here's what I can tell you about CSULB! The university is characterized by exceptional academic programs, outstanding support services, and dedicated faculty, with a diverse student population. Our 322-acre campus provides an inviting collegiate atmosphere as well as state-of-the-art facilities.

Option 2:
Here's what I found about CSULB:

Chosen response: Here's what I can tell you about CSULB! The university is characterized by exceptional academic programs, outstanding support services, and dedicated faculty, with a diverse student population. Our 322-acre campus provides an inviting collegiate atmosphere as well as state-of-the-art facilities.


In [203]:
#uncertainity quotient

import numpy as np

# Mock function to simulate model inference with dropout
def model_inference_with_dropout(model, input_data, num_samples=10):
    """
    Simulate model inference with dropout applied.
    
    :param model: The model used for inference
    :param input_data: The input data for the model
    :param num_samples: Number of Monte Carlo samples
    :return: List of model outputs
    """
    np.random.seed(42)
    outputs = []
    for _ in range(num_samples):
        # Simulate dropout by adding noise to the output
        noise = np.random.normal(0, 0.1, len(input_data))
        output = model(input_data) + noise
        outputs.append(output)
    return outputs

# Mock LLM function
def mock_llm_model(input_data):
    # Placeholder function to represent LLM output
    return np.array([len(data) for data in input_data])

# Simulate multiple inferences with dropout
question = "what are the courses for management information systems program"
context = "contextual data here"  # Replace with actual context

input_data = [context]
num_samples = 10
outputs = model_inference_with_dropout(mock_llm_model, input_data, num_samples=num_samples)

# Calculate uncertainty (standard deviation of outputs)
uncertainty = np.std(outputs)
mean_output = np.mean(outputs)
print(f"Mean Output: {mean_output}, Uncertainty: {uncertainty}")


Mean Output: 20.044806111169873, Uncertainty: 0.06859059302944874


In [204]:
# Threshold for acceptable uncertainty
uncertainty_threshold = 1.0

if uncertainty > uncertainty_threshold:
    print("The model's output is uncertain.")
else:
    print("The model's output is confident.")

# Print the actual generated response (mean of outputs for simplicity)
generated_response = mean_output  # Replace this with actual LLM output interpretation
print(f"Generated Response: {generated_response}")


The model's output is confident.
Generated Response: 20.044806111169873
